# Development Notebook for Analyzing Sandy Twitter Data (from mdredze).

In [2]:
import sys
import os
sys.path.append(os.path.abspath('../'))

import datetime as dt
import json
import numpy as np
import pandas as pd
import pymongo
from twitterinfrastructure.tools import dump, output

import twitterinfrastructure.twitter_sandy as ts
import importlib
importlib.reload(ts)

#os.chdir('../')
print(os.getcwd())

/Users/httran/Documents/projects/twitterinfrastructure


## Test pre-processing tweets.

In [16]:
# test tweet field names
collection = 'tweets_analysis'
tweet_collection = 'tweets'
zones_collection = 'taxi_zones'
fields = ['_id', 'coordinates', 'created_at', 'entities', 'full_text', 
          'id_str', 'place']
db_name = 'sandy'
db_instance = 'mongodb://localhost:27017/'

client = pymongo.MongoClient(db_instance)
db = client[db_name]

zones_iter = db[zones_collection].find()
for zone in zones_iter[0:1]:
    # query tweets within current taxi zone
    query_dict = {
        "coordinates": {
            "$geoWithin": {
                "$geometry": zone['geometry']
            }
        }
    }
    full_tweets = db[tweet_collection].find(query_dict)
    
print(full_tweets.count())
full_tweets = list(full_tweets)

1146


dict_keys(['_id', 'contributors', 'coordinates', 'created_at', 'entities', 'favorite_count', 'favorited', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place', 'retweet_count', 'retweeted', 'source', 'text', 'truncated', 'user'])

dict_keys(['_id', 'contributors', 'coordinates', 'created_at', 'entities', 'extended_entities', 'favorite_count', 'favorited', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place', 'possibly_sensitive', 'retweet_count', 'retweeted', 'source', 'text', 'truncated', 'user'])

dict_keys(['_id', 'contributors', 'coordinates', 'created_at', 'entities', 'favorite_count', 'favorited', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_

In [18]:
for full_tweet in full_tweets[0:5]:
    print(full_tweet.keys())
    print(full_tweet['entities']['hashtags'])
    print(full_tweet['text'])
    print('')

dict_keys(['_id', 'contributors', 'coordinates', 'created_at', 'entities', 'favorite_count', 'favorited', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place', 'retweet_count', 'retweeted', 'source', 'text', 'truncated', 'user'])
[{'indices': [93, 109], 'text': 'reyesenterprise'}, {'indices': [110, 127], 'text': 'workhardplayhard'}]
With my dad Going to delaware with 10 of my dads trucks and 1 crane for emergency generators #reyesenterprise #workhardplayhard

dict_keys(['_id', 'contributors', 'coordinates', 'created_at', 'entities', 'extended_entities', 'favorite_count', 'favorited', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place', 'possibly_sensitive', 'retweet_count', 'retweeted', 'source', 'text', 'truncated', 

In [21]:
# test duplicate key error
# due to tweets found in multiple zones?
collection = 'tweets_analysis'
tweet_collection = 'tweets'
zones_collection = 'taxi_zones'
fields = ['_id', 'coordinates', 'created_at', 'entities', 'full_text', 
          'id_str', 'place']
db_name = 'sandy'
db_instance = 'mongodb://localhost:27017/'

client = pymongo.MongoClient(db_instance)
db = client[db_name]

tweet = db[tweet_collection].find_one({'id_str': '263867240783425536'})
print(tweet)

{'_id': ObjectId('5b10b7000156950952b399d9'), 'contributors': None, 'coordinates': {'coordinates': [-73.95361427, 40.68187311], 'type': 'Point'}, 'created_at': 'Thu Nov 01 04:57:07 +0000 2012', 'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': [{'id': 383956775, 'id_str': '383956775', 'indices': [0, 11], 'name': 'kash', 'screen_name': 'iamkashema'}]}, 'favorite_count': 0, 'favorited': False, 'geo': {'coordinates': [40.68187311, -73.95361427], 'type': 'Point'}, 'id': 263867240783425540, 'id_str': '263867240783425536', 'in_reply_to_screen_name': 'iamkashema', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': 383956775, 'in_reply_to_user_id_str': '383956775', 'is_quote_status': False, 'lang': 'en', 'place': {'attributes': {}, 'bounding_box': {'coordinates': [[[-74.255641, 40.495865], [-73.699793, 40.495865], [-73.699793, 40.91533], [-74.255641, 40.91533]]], 'type': 'Polygon'}, 'contained_within': [], 'country': 'United States',

In [27]:
# test duplicate key error
# query zones intersecting with current tweet (matches 2)
query_dict = {
    "geometry": {
        "$geoIntersects": {
            "$geometry": {
                "type": "Point",
                "coordinates": tweet['coordinates']['coordinates'] 
            }
        }
    }
}
zones = db[zones_collection].find(query_dict)
print(zones.count())
for zone in zones:
    print(zone['properties']['zone'])

2
Clinton Hill
Bedford


In [101]:
# test mongodb and pymongo datetime conversions
# mongodb stores as UTC, but will display in MongoDB Compass based on current
# timezone
analysis_collection = 'tweets_sandy'
db_name = 'sandy'
db_instance = 'mongodb://localhost:27017/'
client = pymongo.MongoClient(db_instance)
db = client[db_name]

tweet = db[analysis_collection].find_one()
utc_time = dt.datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S %z %Y')
print('tweet[created_at]: ' + tweet['created_at'])
print('converted to utc_time:')
print(utc_time)

doc = {}
doc['datetimeUTC'] = utc_time
doc['timestampUNIX'] = int(
    utc_time.replace(tzinfo=dt.timezone.utc).timestamp() * 1000)
#doc['date'] = utc_time.date()
#doc['hour'] = utc_time.hour
print('document to be inserted:')
print(doc)
print('')

collection = 'datetime_test'
db.drop_collection(collection)
db[collection].insert_one(doc)
result1 = db[collection].aggregate([
    {
        "$group": {
            "_id": {
                "month": {"$month": "$datetimeUTC"},
                "day": {"$dayOfMonth": "$datetimeUTC"},
                "year": {"$year": "$datetimeUTC"},
                "hour": {"$hour": "$datetimeUTC"}
            },
            "count": {"$sum": 1}
        }
    }
])
print('aggregation retrieved from collection:')
print(list(result1))

tweet[created_at]: Sun Oct 28 22:42:35 +0000 2012
converted to utc_time:
2012-10-28 22:42:35+00:00
document to be inserted:
{'datetimeUTC': datetime.datetime(2012, 10, 28, 22, 42, 35, tzinfo=datetime.timezone.utc), 'timestampUNIX': 1351464155000}

aggregation retrieved from collection:
[{'_id': {'month': 10, 'day': 28, 'year': 2012, 'hour': 22}, 'count': 1}]


In [6]:
# test create_analysis
import twitterinfrastructure.twitter_sandy as ts
import importlib
importlib.reload(ts)

collection = 'tweets_analysis'
tweet_collection = 'tweets'
nyisozones_collection = 'nyiso_zones'
taxizones_collection = 'taxi_zones'
fields = ['_id', 'coordinates', 'created_at', 'entities', 'text', 
          'id_str', 'place']
db_name = 'sandy'
db_instance = 'mongodb://localhost:27017/'
progressbar = False
overwrite = True
verbose = 1

# import_num, tokens, full_tweets = ts.create_analysis(collection=collection, 
#                                         tweet_collection=tweet_collection, 
#                                         nyisozones_collection=nyisozones_collection,
#                                         taxizones_collection=taxizones_collection,
#                                         fields=fields, db_name=db_name, 
#                                         db_instance=db_instance, 
#                                         progressbar=False, 
#                                         overwrite=overwrite, verbose=verbose)

# connect to db (creates if not exists)
client = pymongo.MongoClient(db_instance)
db = client[db_name]

# ensure that nyisozones_collection and taxizones_collection exist
collections = db.collection_names()
if (nyisozones_collection not in collections) or \
        (taxizones_collection not in collections):
    output('{nyiso} or {taxi} collection not in database. No action '
           'taken.'.format(nyiso=nyisozones_collection,
                           taxi=taxizones_collection))

# process and insert tweets
full_tweets = db[tweet_collection].find()
if progressbar:
    tweets_iter = tqdm(full_tweets, total=full_tweets.count(),
                       desc='tweets', leave=False)
else:
    tweets_iter = full_tweets
full_tweet = tweets_iter[0]
# for full_tweet in tweets_iter:
# remove extra fields
if fields:
    tweet = {field: full_tweet[field] for field in fields}
else:
    tweet = full_tweet

# identify and add nyiso zone, taxi zone, and taxi borough
if tweet['coordinates'] is not None:
    query_dict = {
        "geometry": {
            "$geoIntersects": {
                "$geometry": tweet['coordinates']
            }
        }
    }
    nyiso_zone = db[nyisozones_collection].find_one(query_dict)
    if nyiso_zone:
        tweet['nyiso_zone'] = nyiso_zone['properties']['Zone']
    else:
        tweet['nyiso_zone'] = np.nan

    query_dict = {
        "geometry": {
            "$geoIntersects": {
                "$geometry": tweet['coordinates']
            }
        }
    }
    taxi_zone = db[taxizones_collection].find_one(query_dict)
    if taxi_zone:
        tweet['location_id'] = taxi_zone['properties']['LocationID']
        tweet['borough'] = taxi_zone['properties']['borough']
    else:
        tweet['location_id'] = np.nan
        tweet['borough'] = np.nan
else:
    # fails.append(tweet['id_str'])
    if verbose >= 2:
        output('Tweet skipped due to missing coordinates',
               'create_analysis')



In [34]:
# query analysis collection
client = pymongo.MongoClient(db_instance)
db = client[db_name]
tweets_analysis = db[collection].find()
for tweet in tweets_analysis:
    print(tweet['full_text'])
    print(tweet['tokens'])
    print(tweet['timestamp'])
    print('')
    
print(tokens)

@NOT_savinHOES Not r yu upp
['r', 'yu', 'upp']
1350882000.0

Who's up?
["who'"]
1350882000.0

@augustushazel idk I'm just ugly or annoying or something
['idk', "i'm", 'ugli', 'annoy', 'someth']
1350882000.0

@InYurMomsCovers your cool thou. You seem like one of those girls that you can talk about anything to. Not too many are like that
['cool', 'thou', 'seem', 'like', 'one', 'girl', 'talk', 'anyth', 'mani', 'like']
1350882001.0

"I suppose she has an appropriate costume for every activity..." #ilovemaggiesmith #downtonseasonthree
['suppos', 'appropri', 'costum', 'everi', 'activ', '...', '#ilovemaggiesmith', '#downtonseasonthre']
1350882000.0

Hit and Run is so sad..
['hit', 'run', 'sad', '..']
1350882000.0

10/22 @ 01:00 - Temperature 46.6F. Wind 1.5mph SW. Gust 3.8mph. Barometer 30.06in, Steady. Rain today 0.00in. Humidity 92%.
['10/22', '01:00', 'temperatur', '46.6', 'F', 'wind', '1.5', 'mph', 'SW', 'gust', '3.8', 'mph', 'baromet', '30.06', 'steadi', 'rain', 'today', '0.00', 'humid',

In [2]:
# test dump
dump(['123', '456'], func_name='test')

True

## Test tweet keyword filters.

In [4]:
# query tweets containing specified tokens
db_instance = 'mongodb://localhost:27017/'
db_name = 'sandy_test'
collection = 'tweets_analysis_test'

tokens = ['gust', 'humid', 'idk']
tweets = ts.query_keyword(tokens=tokens, 
                          collection=collection, db_name=db_name, 
                          db_instance=db_instance, verbose=1)

for tweet in tweets:
    print(tweet['id_str'])
    print(tweet['full_text'])
    print(tweet['coordinates'])
    print(tweet.keys())
    print('')

2018-06-01 09:06:44 : Started query.

2018-06-01 09:06:44 : Finished query. Returned 2 tweets.

260244089985957888
@augustushazel idk I'm just ugly or annoying or something
{'coordinates': [-80.08961896, 42.09464892], 'type': 'Point'}
dict_keys(['_id', 'coordinates', 'created_at', 'entities', 'full_text', 'id_str', 'place', 'location_id', 'borough', 'tokens'])

260244093119102977
10/22 @ 01:00 - Temperature 46.6F. Wind 1.5mph SW. Gust 3.8mph. Barometer 30.06in, Steady. Rain today 0.00in. Humidity 92%.
{'coordinates': [-75.68444444, 39.695], 'type': 'Point'}
dict_keys(['_id', 'coordinates', 'created_at', 'entities', 'full_text', 'id_str', 'place', 'location_id', 'borough', 'tokens'])



In [111]:
# query tweets containing specified hashtags
db_instance = 'mongodb://localhost:27017/'
db_name = 'sandy_test'
collection = 'tweets_analysis_test'

hashtags = ['downtonseasonthree', 'sandy']
tweets = ts.query_keyword(hashtags=hashtags, 
                          collection=collection, db_name=db_name, 
                          db_instance=db_instance, verbose=1)

for tweet in tweets:
    print(tweet['full_text'])
    print(tweet['entities']['hashtags'])
    print('')

2018-05-30 16:27:12 : Started query.

2018-05-30 16:27:12 : Finished query. Returned 1 tweets.

"I suppose she has an appropriate costume for every activity..." #ilovemaggiesmith #downtonseasonthree
[{'indices': [65, 82], 'text': 'ilovemaggiesmith'}, {'indices': [83, 102], 'text': 'downtonseasonthree'}]



In [112]:
# query tweets containing specified tokens or hashtags
db_instance = 'mongodb://localhost:27017/'
db_name = 'sandy_test'
collection = 'tweets_analysis_test'

tokens = ['gust', 'humid', 'idk']
hashtags = ['test', 'downtonseasonthree']
tweets = ts.query_keyword(tokens=tokens, hashtags=hashtags, 
                          collection=collection, db_name=db_name, 
                          db_instance=db_instance, verbose=1)

for tweet in tweets:
    print(tweet['full_text'])
    print('')

2018-05-30 16:27:33 : Started query.

2018-05-30 16:27:33 : Finished query. Returned 3 tweets.

@augustushazel idk I'm just ugly or annoying or something

"I suppose she has an appropriate costume for every activity..." #ilovemaggiesmith #downtonseasonthree

10/22 @ 01:00 - Temperature 46.6F. Wind 1.5mph SW. Gust 3.8mph. Barometer 30.06in, Steady. Rain today 0.00in. Humidity 92%.



## Test group by summaries

In [40]:
# test borough_day summary
# can check datetime tz by finding first tweet in tweets_sandy
title = 'sandy_test'
analysis_collection = 'tweets_sandy'
db_name = 'sandy'
db_instance = 'mongodb://localhost:27017/'

client = pymongo.MongoClient(db_instance)
db = client[db_name]

pipeline = [
    {
        "$group": {
            "_id": {
                "year": {"$year": "$datetimeUTC"},
                "month": {"$month": "$datetimeUTC"},
                "day": {"$dayOfMonth": "$datetimeUTC"},
                "hour": {"$hour": "$datetimeUTC"},
                "borough": "$borough"
            },
            "datetimeUTC": {"$min": {"$dateFromParts": {
                "year": {"$year": "$datetimeUTC"},
                "month": {"$month": "$datetimeUTC"},
                "day": {"$dayOfMonth": "$datetimeUTC"},
                "hour": {"$hour": "$datetimeUTC"}
            }}},
            "borough": {"$first": "$borough"},
            "count": {"$sum": 1}
        }
    },
    {"$sort": {"datetimeUTC": 1, "_id.borough": 1}}
]
groups = list(db[analysis_collection].aggregate(pipeline))

# convert to dataframe
# groups = json.loads(df.to_json(orient='records', date_format='iso'))
df = pd.DataFrame(groups)
df['datetimeUTC'] = [datetime.tz_localize(tz='UTC') for datetime 
                     in df['datetimeUTC']]
print(df.head())

# groups = list(db[analysis_collection].aggregate(pipeline))

                                                 _id    borough  count  \
0  {'year': 2012, 'month': 10, 'day': 22, 'hour':...   Brooklyn      1   
1  {'year': 2012, 'month': 10, 'day': 22, 'hour':...   Brooklyn      1   
2  {'year': 2012, 'month': 10, 'day': 22, 'hour':...  Manhattan      1   
3  {'year': 2012, 'month': 10, 'day': 22, 'hour':...  Manhattan      1   
4  {'year': 2012, 'month': 10, 'day': 22, 'hour':...     Queens      1   

                datetimeUTC  
0 2012-10-22 13:00:00+00:00  
1 2012-10-22 15:00:00+00:00  
2 2012-10-22 15:00:00+00:00  
3 2012-10-22 16:00:00+00:00  
4 2012-10-22 23:00:00+00:00  


In [ ]:
# test datetime group summaries with pandas and tzone conversion

tweet_count_filter = 5  # by day
# startdate = pd.Timestamp(2012, 10, 28, 0, 0, 0)  # inclusive
# enddate = pd.Timestamp(2012, 11, 3, 0, 0, 0)  # exclusive
startdate = pd.Timestamp('2012-10-28 00:00:00', 
                         tz='America/New_York')  # inclusive
enddate = pd.Timestamp('2012-11-03 00:00:00', 
                       tz='America/New_York')  # exclusive

# load sandy-related tweets filtered by dates
df_sandy = ts.mongod_to_df({}, collection='tweets_sandy')
df_sandy['datetimeUTC'] = [datetime.tz_localize(tz='UTC') for datetime
                           in df_sandy['datetimeUTC']]
df_sandy['datetime'] = [datetime.tz_convert('America/New_York') for datetime
                        in df_sandy['datetimeUTC']]
df_sandy = df_sandy.rename(columns={'location_id': 'zone'})
df_sandy = df_sandy[['datetime', 'zone', 'tokens']]
df_sandy = df_sandy.set_index('datetime')
df_sandy = df_sandy.sort_index()
df_sandy = df_sandy.loc[startdate:enddate]
print('[min, max] sandy tweets datetime: [' + 
      str(min(df_sandy.index.get_level_values('datetime'))) + ', ' + 
      str(max(df_sandy.index.get_level_values('datetime'))) + '].')
# group by datetime and zone
df_sandy = df_sandy.reset_index()
df_sandy['datetime'] = df_sandy['datetime'].dt.date
df_sandy_group = df_sandy.groupby(['zone', 'datetime']).count()
df_sandy_group = df_sandy_group.rename(columns={'tokens': 'sandy-tweets'})
df_sandy_group = df_sandy_group[df_sandy_group['sandy-tweets'] >= tweet_count_filter] 
print('[min, max] sandy tweets: [' + 
      str(np.nanmin(df_sandy_group['sandy-tweets'])) + ', ' +
      str(np.nanmax(df_sandy_group['sandy-tweets'])) + '].')
print('')
df_sandy_group.head()

## Test heatmap

In [40]:
tweet_count_filter = 5
#startdate = dt.datetime(2012, 10, 21, 0, 0, 0).isoformat()
#enddate = dt.datetime(2012, 10, 28, 0, 0, 0).isoformat()
startdate = dt.datetime(2012, 10, 28, 0, 0, 0).isoformat()
enddate = dt.datetime(2012, 11, 4, 0, 0, 0).isoformat()
query = {
    "count": {"$gte": tweet_count_filter},
    "datetimeUTC": {"$lte": enddate, "$gte": startdate}
}
print(query)
df = ts.mongod_to_df(query, collection='borough_day_sandy')

#daterange = ['10/22/2012', '10/27/2012']
daterange = ['10/28/2012', '11/03/2012']
#daterange = None

# update dtypes and columns 
df['date'] = pd.to_datetime(df['datetimeUTC']).dt.date
df = df[['date', 'borough', 'count']]

# build full dataframe (all dates and all boroughs initialized with nans)
if daterange:
    dates = pd.date_range(start=daterange[0], end=daterange[1]).tolist()
    dates = [pd.Timestamp.to_pydatetime(date).date() for date in dates]
else:
    dates = df['date'].unique()
#print(dates)
boroughs = sorted(df['borough'].unique())
df_proc = pd.DataFrame({'date': [], 'borough': [], 'count': []})
for date in dates:
    for borough in boroughs:
        df_temp = pd.DataFrame({'date': date,
                                'borough': borough,
                                'count': [np.nan]})
        df_proc = df_proc.append(df_temp, ignore_index=True)

# get matching indexes in df_proc of available data in df
proc_indexes = [df_proc.index[(df_proc['date'] == date) & (
                df_proc['borough'] == borough)].tolist()[0]
                for date, borough in zip(list(df['date']),
                                         list(df['borough']))]

# update df_proc with available data in df
df.index = proc_indexes
df_proc.loc[proc_indexes, ['count']] = df['count']

# reformat and rename columns
df_proc['date'] = df_proc['date'].apply(lambda x: x.strftime('%m-%d'))

# pivot dataframe for heat map visualization
df_pivot = df_proc.pivot('borough', 'date', 'count')

print(df[0:5])
print(df_proc[0:5])
print('')
print(df[-10::])
print(df_proc[-10::])

{'count': {'$gte': 5}, 'datetimeUTC': {'$lte': '2012-11-04T00:00:00', '$gte': '2012-10-28T00:00:00'}}
         date    borough  count
0  2012-10-28      Bronx    146
1  2012-10-28   Brooklyn    656
2  2012-10-28        EWR     28
3  2012-10-28  Manhattan   1436
4  2012-10-28     Queens    498
    date    borough   count
0  10-28      Bronx   146.0
1  10-28   Brooklyn   656.0
2  10-28        EWR    28.0
3  10-28  Manhattan  1436.0
4  10-28     Queens   498.0

          date        borough  count
25  2012-11-01       Brooklyn    226
26  2012-11-01            EWR      6
27  2012-11-01      Manhattan    438
28  2012-11-01         Queens    136
29  2012-11-01  Staten Island     52
30  2012-11-02          Bronx     13
31  2012-11-02       Brooklyn     53
33  2012-11-02      Manhattan    105
34  2012-11-02         Queens     30
35  2012-11-02  Staten Island     41
     date        borough  count
32  11-02            EWR    NaN
33  11-02      Manhattan  105.0
34  11-02         Queens   30.0
35

## Code for test scripts

In [73]:
# test create_tweet_analysis
import sys
import os
sys.path.append(os.path.abspath('../'))

import twitterinfrastructure.twitter_sandy as ts
import importlib
importlib.reload(ts)



[260244087901413376, 260244088203403264, 260244088161439744, 260244088819945472, 260244089080004609, 260244089985957888, 260244092527706112, 260244093119102977, 260244093257515008, 260244094939439105]


True

# Extras.